In [1]:
!wget https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf


--2025-03-07 10:48:05--  https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf
Resolving huggingface.co (huggingface.co)... 18.164.174.55, 18.164.174.23, 18.164.174.17, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.55|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/41/c8/41c860f65b01de5dc4c68b00d84cead799d3e7c48e38ee749f4c6057776e2e9e/5d99003e395775659b0dde3f941d88ff378b2837a8dc3a2ea94222ab1420fad3?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27Phi-3-mini-4k-instruct-fp16.gguf%3B+filename%3D%22Phi-3-mini-4k-instruct-fp16.gguf%22%3B&Expires=1741348086&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MTM0ODA4Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzQxL2M4LzQxYzg2MGY2NWIwMWRlNWRjNGM2OGIwMGQ4NGNlYWQ3OTlkM2U3YzQ4ZTM4ZWU3NDlmNGM2MDU3Nzc2ZTJlOWUvNWQ5OTAwM2UzOTU3NzU2NTliMGRkZTNmOTQxZ

In [2]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.1/415.1 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.40
    Uninstalling langchain-core-0.3.40:
      Successfully uninstalled langchain-core-0.3.40
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.19
    Uninstalling langchain-0.3.19:
      Successfully uninstalled langchain-0.3.19


In [3]:
pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 MB 9.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.1 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.7-cp311-cp311-linux_x86_64.whl size=4601112 sha256=f4532510067a62999ebf407af203e8dc0544275fe7cd56ff1623cae6bbf5ad20
  Stored in directory: /root/.cache/pip/wheels/eb/82/79/ac77fcd49324b75ae6aa18e63a87cf9da4371a57e2cdc8dc03
Successfully built llama-cpp-python


In [4]:
from langchain import LlamaCpp

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="Phi-3-mini-4k-instruct-fp16.gguf",
    n_gpu_layers=-1,
    max_tokens=500,
    n_ctx=2048, # context window size - how many previous tokens the model remembers.
    seed=42,
    #temperature=0.7,
    #top_k=40,
    verbose=False
)

llama_init_from_model: n_batch is less than GGML_KQ_MASK_PAD - increasing to 32
llama_init_from_model: n_ctx_per_seq (2048) < n_ctx_train (4096) -- the full capacity of the model will not be utilized


LlamaCpp (Llama.cpp) is a lightweight and efficient C++ implementation of Llama-based models (Meta's LLaMA models and other GGUF models like Phi-3).
It enables running large language models locally on CPUs or GPUs without relying on cloud APIs like OpenAI.
It is optimized for low-memory usage, making it possible to run LLMs on personal computers.

In [8]:
llm.invoke("Hi! My name is Maarten. What is 1 + 1?")

''

The invoke() method in LangChain is used to send a prompt to an LLM and get a response synchronously (i.e., waiting for the model to generate a reply before moving on).

Unfortunately, we get no output! Phi-3 requires
a specific prompt template. Compared to our examples with transformers, we will
need to explicitly use a template ourselves. Instead of copy-pasting this template each
time we use Phi-3 in LangChain, we can use one of LangChain’s core functionalities,
namely “chains.”

## Chains

In [6]:
from langchain import PromptTemplate

# Create a prompt template with the "input_prompt" variable
template = """<s><|user|>
{input_prompt}<|end|>
<|assistant|>"""
prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt"]
)

In [7]:
basic_chain = prompt | llm
#basic_chain = prompt.pipe(llm)

In [8]:
# Use the chain
basic_chain.invoke(
    {
        "input_prompt": "Hi! My name is Maarten. What is 1 + 1?",
    }
)

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


' Hello Maarten! The answer to 1 + 1 is 2.'

### Multiple Chains

In [9]:
from langchain import LLMChain

# Create a chain for the title of our story
template = """<s><|user|>
Create a title for a story about {summary}. Only return the title.<|end|>
<|assistant|>"""
title_prompt = PromptTemplate(template=template, input_variables=["summary"])
title = LLMChain(llm=llm, prompt=title_prompt, output_key="title")

<ipython-input-9-61dd782c6da9>:8: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  title = LLMChain(llm=llm, prompt=title_prompt, output_key="title")


In [10]:
title.invoke({"summary": "a girl that lost her mother"})

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'summary': 'a girl that lost her mother',
 'title': ' "Whispers of a Mother\'s Love: A Tale of Healing and Resilience"'}

In [11]:
# Create a chain for the character description using the summary and title
template = """<s><|user|>
Describe the main character of a story about {summary} with the title {title}. Use only two sentences.<|end|>
<|assistant|>"""
character_prompt = PromptTemplate(
    template=template, input_variables=["summary", "title"]
)
character = LLMChain(llm=llm, prompt=character_prompt, output_key="character")

In [13]:
# Create a chain for the story using the summary, title, and character description
template = """<s><|user|>
Create a story about {summary} with the title {title}. The main character is:
{character}. Only return the story and it cannot be longer than one paragraph.
<|end|>
<|assistant|>"""
story_prompt = PromptTemplate(
template=template, input_variables=["summary", "title", "character"]
)
story = LLMChain(llm=llm, prompt=story_prompt, output_key="story")

In [14]:
# Combine all three components to create the full chain
llm_chain = title | character | story

In [15]:
llm_chain.invoke("a girl that lost her mother")


/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'summary': 'a girl that lost her mother',
 'title': ' "Whispers of a Farewell: The Journey Through Loss"',
 'character': " The protagonist, a resilient and compassionate 14-year-old girl named Sophia, faces the unimaginable pain of losing her mother to cancer. With inner strength and an indomitable spirit, she navigates through grief while seeking solace in cherished memories with her beloved mother.\n\nSophia's journey is marked by a deep bond shared with her mother, who instilled love for learning and adventure into her heart; as the news of her mother's passing arrives, Sophia struggles to come to terms with this monumental loss but finds comfort in the timeless whispers of love that echo through their memories. With each step forward on her path of healing, she discovers resilience within herself and learns valuable lessons about life, family, and hope, ultimately growing into a remarkable young woman who honors her mother's memory with grace and determination.",
 'story': ' "Whis

## Memory: Helping LLMs to Remember Conversations

To make LLM models stateful, we can add specific types of memory to the chain
that we created earlier. Two common methods for
helping LLMs to remember conservations are:

• Conversation buffer

• Conversation summary

### Conversation *Buffer*

One of the most intuitive forms of giving LLMs memory is simply reminding them
exactly what has happened in the past. We can achieve
this by copying the full conversation history and pasting that into our prompt.

In [16]:
# Create an updated prompt template to include a chat history
template = """<s><|user|>Current conversation:{chat_history}

{input_prompt}<|end|>
<|assistant|>"""

prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt", "chat_history"]
)

In [17]:
from langchain.memory import ConversationBufferMemory

# Define the type of Memory we will use
memory = ConversationBufferMemory(memory_key="chat_history")

# Chain the LLM, Prompt, and Memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

<ipython-input-17-9823403f20ac>:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history")


In [18]:
# Generate a conversation and ask a basic question
llm_chain.invoke({"input_prompt": "Hi! My name is Maarten. What is 1 + 1?"})

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'Hi! My name is Maarten. What is 1 + 1?',
 'chat_history': '',
 'text': " The answer to 1 + 1 is 2. It's a basic arithmetic operation where you add one unit to another, resulting in two units.\n\nHere’s a simple breakdown:\n- You start with the number 1.\n- Then you add another number 1 to it.\n- The sum of these numbers (1+1) equals 2."}

In [19]:
# Does the LLM remember the name we gave it?
llm_chain.invoke({"input_prompt": "What is my name?"})

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'What is my name?',
 'chat_history': "Human: Hi! My name is Maarten. What is 1 + 1?\nAI:  The answer to 1 + 1 is 2. It's a basic arithmetic operation where you add one unit to another, resulting in two units.\n\nHere’s a simple breakdown:\n- You start with the number 1.\n- Then you add another number 1 to it.\n- The sum of these numbers (1+1) equals 2.",
 'text': " Your name is Maarten. It seems you already know the answer to 1 + 1, which is 2. If there's anything else you need help with or any other questions, feel free to ask!"}

### ConversationBufferMemoryWindow

In our previous example, we essentially created a chatbot. You could talk to it and
it remembers the conversation you had thus far. However, as the size of the conversation
grows, so does the size of the input prompt until it exceeds the token limit.
One method of minimizing the context window is to use the last k conversations
instead of maintaining the full chat history. In LangChain, we can use Conversation
BufferWindowMemory to decide how many conversations are passed to the input
prompt




In [20]:
from langchain.memory import ConversationBufferWindowMemory

# Retain only the last 2 conversations in memory
memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history")

# Chain the LLM, Prompt, and Memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

<ipython-input-20-046ef635f261>:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history")


In [21]:
# Ask two questions and generate two conversations in its memory
llm_chain.invoke({"input_prompt":"Hi! My name is Maarten and I am 33 years old. What is 1 + 1?"})
llm_chain.invoke({"input_prompt":"What is 3 + 3?"})

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'What is 3 + 3?',
 'chat_history': "Human: Hi! My name is Maarten and I am 33 years old. What is 1 + 1?\nAI:  Hello Maarten! It's a pleasure to meet you. To answer your question, 1 + 1 equals 2.",
 'text': " Hello Maarten! It's a pleasure to meet you. To answer your question, 3 + 3 equals 6."}

In [24]:
# Check whether it knows the name we gave it
llm_chain.invoke({"input_prompt":"What is my name?"})

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'What is my name?',
 'chat_history': "Human: What is 3 + 3?\nAI:  Hello Maarten! It's a pleasure to meet you. To answer your question, 3 + 3 equals 6.\nHuman: What is my age?\nAI:  Hello Maarten! As an AI, I don't have access to personal data like age unless you share it during our conversation. From the information you provided earlier, you mentioned that you are 33 years old.\n\nHowever, if this is a new context and not from your previous message, please provide me with the relevant details so I can assist you accordingly.",
 'text': " AI: Hello! I don't have access to personal information unless it's shared with me in our conversation. You mentioned your name as Maarten earlier, so based on that, you referred to yourself as Maarten. However, if this is a new context or not from the previous message, please let me know your actual name for further assistance."}

In [25]:
# Check whether it knows the age we gave it
llm_chain.invoke({"input_prompt":"What is my age?"})

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'What is my age?',
 'chat_history': "Human: What is my age?\nAI:  Hello Maarten! As an AI, I don't have access to personal data like age unless you share it during our conversation. From the information you provided earlier, you mentioned that you are 33 years old.\n\nHowever, if this is a new context and not from your previous message, please provide me with the relevant details so I can assist you accordingly.\nHuman: What is my name?\nAI:  AI: Hello! I don't have access to personal information unless it's shared with me in our conversation. You mentioned your name as Maarten earlier, so based on that, you referred to yourself as Maarten. However, if this is a new context or not from the previous message, please let me know your actual name for further assistance.",
 'text': " I'm unable to determine your age without recent information as my knowledge is static and does not update in real-time. If you provide the current year, I can help calculate an estimated age fo

The LLM indeed has no access to our age since that was not retained in the chat
history.
Although this method reduces the size of the chat history, it can only retain the last
few conversations, which is not ideal for lengthy conversations. Let’s explore how we
can summarize the chat history instead.

### Conversation Summary

The summarization process is enabled by another LLM that is given the conversation
history as input and asked to create a concise summary. A nice advantage of using an
external LLM is that we are not confined to using the same LLM during conversation.

This means that whenever we ask the LLM a question, there are two calls:

• The user prompt

• The summarization prompt

To use this in LangChain, we first need to prepare a summarization template that we
will use as the summarization prompt:



In [26]:
# Create a summary prompt template
summary_prompt_template = """<s><|user|>Summarize the conversations and update with the new lines.

Current summary:
{summary}

new lines of conversation:
{new_lines}

New summary:<|end|>
<|assistant|>"""
summary_prompt = PromptTemplate(
    input_variables=["new_lines", "summary"],
    template=summary_prompt_template
)

Using ConversationSummaryMemory in LangChain is similar to what we did with the
previous examples. The main difference is that we additionally need to supply it
with an LLM that performs the summarization task. Although we use the same LLM
for both summarizing and user prompting, you could use a smaller LLM for the
summarization task to speed up computation:

In [27]:
from langchain.memory import ConversationSummaryMemory

# Define the type of memory we will use
memory = ConversationSummaryMemory(
    llm=llm,
    memory_key="chat_history",
    prompt=summary_prompt
)

# Chain the LLM, prompt, and memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

<ipython-input-27-905e44fca19a>:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(


In [28]:
# Generate a conversation and ask for the name
llm_chain.invoke({"input_prompt": "Hi! My name is Maarten. What is 1 + 1?"})
llm_chain.invoke({"input_prompt": "What is my name?"})

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'What is my name?',
 'chat_history': ' Maarten introduced himself and asked the AI to solve a basic arithmetic operation, 1 + 1. The AI responded by stating that the sum of 1 + 1 equals 2, explaining it as adding one unit to another to result in two units.',
 'text': ' Your name is not mentioned in this conversation. Maarten introduced himself, but there was no mention of your name.'}

In [29]:
# Check whether it has summarized everything thus far
llm_chain.invoke({"input_prompt": "What was the first question I asked?"})

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'What was the first question I asked?',
 'chat_history': ' Maarten introduced himself and inquired about a basic arithmetic operation, asking the AI to solve 1 + 1. The AI correctly answered that 1 + 1 equals 2 by explaining it as adding one unit to another, resulting in two units. When asked about his name, the AI clarified that no name was mentioned within this conversation, and only Maarten had introduced himself.',
 'text': ' The first question you asked was, "solve 1 + 1."'}

In [30]:
# Check what the summary is thus far
memory.load_memory_variables({})

{'chat_history': ' Maarten introduced himself to the AI and asked it to solve a basic arithmetic operation, specifically 1 + 1. The AI correctly responded that 1 + 1 equals 2 by explaining it as adding one unit to another, resulting in two units. Later, when asked about his name, the AI clarified that no name was mentioned within this conversation, and only Maarten had introduced himself. When queried about the first question he asked, the human confirmed it as "solve 1 + 1."'}

| Memory Type                      | Pros                                                                 | Cons                                                                 |
|----------------------------------|---------------------------------------------------------------------|---------------------------------------------------------------------|
| **Conversation Buffer**          | - Easiest implementation  <br> - Ensures no information loss within context window | - Slower generation speed as more tokens are needed  <br> - Only suitable for large-context LLMs  <br> - Larger chat histories make information retrieval difficult |
| **Windowed Conversation Buffer** | - Large-context LLMs are not needed unless chat history is large  <br> - No information loss over the last k interactions | - Only captures the last k interactions  <br> - No compression of the last k interactions |
| **Conversation Summary**         | - Captures the full history  <br> - Enables long conversations  <br> - Reduces tokens needed to capture full history | - An additional call is necessary for each interaction  <br> - Quality is reliant on the LLM’s summarization capabilities |


# Agents: Creating a Systems of LLMs

In [ ]:
import os
from langchain_openai import ChatOpenAI

# Load OpenAI's LLMs with LangChain
os.environ["OPENAI_API_KEY"] = "MY_KEY"
openai_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [ ]:
# Create the ReAct template
react_template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate(
    template=react_template,
    input_variables=["tools", "tool_names", "input", "agent_scratchpad"]
)

In [ ]:
from langchain.agents import load_tools, Tool
from langchain.tools import DuckDuckGoSearchResults

# You can create the tool to pass to an agent
search = DuckDuckGoSearchResults()
search_tool = Tool(
    name="duckduck",
    description="A web search engine. Use this to as a search engine for general queries.",
    func=search.run,
)

# Prepare tools
tools = load_tools(["llm-math"], llm=openai_llm)
tools.append(search_tool)

In [ ]:
from langchain.agents import AgentExecutor, create_react_agent

# Construct the ReAct agent
agent = create_react_agent(openai_llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [ ]:
# What is the Price of a MacBook Pro?
agent_executor.invoke(
    {
        "input": "What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD?"
    }
)